# Custom Data Creation

## Importing the libraries

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


## Importing the dataset

In [2]:
df = pd.read_excel("1_updated_data/1_updated_data.xlsx")


In [3]:
print(df.head())

                Current_Role  Zip Code  Employee_ID  Employee Annual Salary$  \
0   Senior Software Engineer     94103           57                   130800   
1   Associate Data Scientist     94111           58                    74250   
2  Associate Product Manager     10009           59                   125850   
3           Business Analyst     60605           60                    84750   
4    Chief Operating Officer      2101           61                   300000   

   Monthly Medical contribution$  Monthly Dental Contribution$  \
0                            130                            35   
1                            105                            30   
2                            130                            35   
3                            105                            30   
4                            230                            55   

   Monthly Vision Contribution$  Bonus $  Years_Of_Service    Department  \
0                            19    13865      

## Merging the dataset

In [49]:
# Merge on SAP Personnel # and Employee_ID
df_merged = pd.merge(df_s4, df_workday, left_on='SAP Personnel #', right_on='Employee_ID', how='inner')

# Drop both ID columns since they are just identifiers
df_merged.drop(columns=['SAP Personnel #'], errors='ignore', inplace=True)

In [50]:
print(df_merged.head())

                Current_Role  Birthdate  Zip Code  Employee_ID  \
0   Senior Software Engineer 1991-05-19     94103           57   
1   Associate Data Scientist 1996-03-23     94111           58   
2  Associate Product Manager 1993-09-25     10009           59   
3           Business Analyst 1994-03-03     60605           60   
4    Chief Operating Officer 1984-01-13      2101           61   

   Employee Annual Salary$  Monthly Medical contribution$  \
0                   130800                            130   
1                    74250                            105   
2                   125850                            130   
3                    84750                            105   
4                   300000                            230   

   Monthly Dental Contribution$  Monthly Vision Contribution$  Bonus $  \
0                            35                            19    13865   
1                            30                            17     4344   
2             

In [51]:
print(df_merged.shape)

(100, 15)


## Feature Engineering

In [52]:
df_merged['Last_Promoted'] = pd.to_datetime(df_merged['Last_Promoted'])
df_merged['Birthdate'] = pd.to_datetime(df_merged['Birthdate'])

today = pd.Timestamp.today()

# Calculate years since last promotion using days / 365.25
df_merged['Years_Since_Last_Promotion'] = (today - df_merged['Last_Promoted']).dt.days / 365.25

# Calculate age as integer years
df_merged['age'] = ((today - df_merged['Birthdate']).dt.days / 365.25).astype(int)


# Renaming DataFrame for feature-engineered data
df_featured = df_merged.copy()

# Optionally drop original columns
df_featured.drop(columns=['Last_Promoted', 'Birthdate'], inplace=True)

print(df_featured[['Years_Since_Last_Promotion', 'age']].head())

   Years_Since_Last_Promotion  age
0                    2.800821   34
1                    1.563313   29
2                    1.245722   31
3                    2.297057   31
4                    5.782341   41


In [53]:
print(df_featured.head())

                Current_Role  Zip Code  Employee_ID  Employee Annual Salary$  \
0   Senior Software Engineer     94103           57                   130800   
1   Associate Data Scientist     94111           58                    74250   
2  Associate Product Manager     10009           59                   125850   
3           Business Analyst     60605           60                    84750   
4    Chief Operating Officer      2101           61                   300000   

   Monthly Medical contribution$  Monthly Dental Contribution$  \
0                            130                            35   
1                            105                            30   
2                            130                            35   
3                            105                            30   
4                            230                            55   

   Monthly Vision Contribution$  Bonus $  Years_Of_Service    Department  \
0                            19    13865      

In [54]:
print(df_featured.columns)

Index(['Current_Role', 'Zip Code', 'Employee_ID', 'Employee Annual Salary$',
       'Monthly Medical contribution$', 'Monthly Dental Contribution$',
       'Monthly Vision Contribution$', 'Bonus $', 'Years_Of_Service',
       'Department', 'Gender', 'Employee HR rate', '# of Hours per week',
       'Years_Since_Last_Promotion', 'age'],
      dtype='object')


In [55]:
print(df_featured.shape)

(100, 15)


## Create Custom Dependent Column

In [56]:
# Create a boolean Series for condition
condition = df_featured['Years_Since_Last_Promotion'] > 5

# Group by that boolean condition
grouped = df_featured.groupby(condition)


In [57]:
print(grouped.size())

Years_Since_Last_Promotion
False    89
True     11
dtype: int64


In [58]:
df_featured['left'] = (df_featured['Years_Since_Last_Promotion'] > 5).astype(int)

# Renaming DataFrame for updated-engineered data
df_updated = df_featured.copy()

In [59]:
print(df_updated[['Years_Since_Last_Promotion', 'left']].head())

   Years_Since_Last_Promotion  left
0                    2.800821     0
1                    1.563313     0
2                    1.245722     0
3                    2.297057     0
4                    5.782341     1


## Export Updated Excel

In [60]:
# Create the folder if it doesn't exist
os.makedirs("1_updated_data", exist_ok=True)

# Save the Excel file inside the folder
df_updated.to_excel("1_updated_data/updated_data.xlsx", index=False)
